In [144]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [145]:
url = 'https://www.britannica.com/art/Grammy-Award'
response = requests.get(url)

In [146]:
# save the HTML to file
with open ('Grammy-Award.html', mode = 'wb') as file:
    file.write(response.content)

In [147]:
# work with html in memory
soup = BeautifulSoup(response.content, 'html.parser')

In [148]:
# variable definations
table_dict = {'year':[], 'record of the year':[], 'song of the year':[], 'album of the year':[], 'best new artist': []}
keys=['year', 'record of the year', 'song of the year', 'album of the year', 'best new artist']

for a in range(len(soup.table.find_all('tr'))):
    # narrow down on elements in each row of the table
    tr = soup.table.find_all('tr')[a]
    

    for i in range(len(tr.find_all('td'))):
        # narrow down on element in each column
        content = tr.find_all('td')[i].text
        
        #append into the dictionary
        table_dict[keys[i]].append(content)
    
   

In [149]:
table_dict['year'] = table_dict['year'][3:]

In [150]:
table_df = pd.DataFrame.from_dict(table_dict)

In [151]:
table_df.iloc[11,-1]

'\nCrosby, Stills and Nash\n'

#### cleaning

In [152]:
splitted_table_df = pd.DataFrame.from_dict(table_dict['year'])
key = keys[1:]
key_2 = ['artist name record', 'artist name song', 'artist name album', 'artist name best']
for i in range(len(key)):
    splitted_table_df[[key[i], key_2[i]]] = table_df[key[i]].str.rsplit(",",n= 1, expand = True)
    

In [153]:
splitted_table_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   0                   65 non-null     object
 1   record of the year  65 non-null     object
 2   artist name record  65 non-null     object
 3   song of the year    65 non-null     object
 4   artist name song    62 non-null     object
 5   album of the year   65 non-null     object
 6   artist name album   65 non-null     object
 7   best new artist     65 non-null     object
 8   artist name best    1 non-null      object
dtypes: object(9)
memory usage: 4.7+ KB


In [154]:
splitted_table_df[splitted_table_df['artist name best'].notnull()]

,0,record of the year,artist name record,song of the year,artist name song,album of the year,artist name album,best new artist,artist name best
11,\n 1969\n,"\n ""Aquarius/Let the Sunshine In",""" the 5th Dimension\n","\n ""Games People Play",""" Joe South\n","\n Blood, Sweat and Tears, Blood",Sweat and Tears\n,\nCrosby,Stills and Nash\n


In [155]:
splitted_table_df.rename(columns = {0:'year'}, inplace = True)
splitted_table_df.drop('artist name best', axis = 1, inplace = True)
splitted_table_df.fillna("", inplace = True)
splitted_table_df.iloc[11,-1] = 'Crosby, Stills and Nash'

In [156]:
column_name = splitted_table_df.columns.to_list()
for i in column_name:
    splitted_table_df[i] = splitted_table_df[i].apply(lambda x: re.sub(r'"', '', x).strip())

In [157]:
splitted_table_df

,year,record of the year,artist name record,song of the year,artist name song,album of the year,artist name album,best new artist
0,1958,Nel blu dipinto di blu (Volaré),Domenico Modugno,Nel blu dipinto di blu (Volaré),Domenico Modugno,The Music from Peter Gunn,Henry Mancini,not awarded
1,1959,Mack the Knife,Bobby Darin,The Battle of New Orleans,Jimmy Driftwood,Come Dance with Me,Frank Sinatra,Bobby Darin
2,1960,The Theme from 'A Summer Place,' Percy Faith,Theme from Exodus,Ernest Gold,The Button-Down Mind of Bob Newhart,Bob Newhart,Bob Newhart
3,1961,Moon River,Henry Mancini,Moon River,Henry Mancini,Judy at Carnegie Hall,Judy Garland,Peter Nero
4,1962,I Left My Heart in San Francisco,Tony Bennett,What Kind of Fool Am I?***,,The First Family,Vaughn Meader,Robert Goulet
...,...,...,...,...,...,...,...,...
60,2018,This Is America,Childish Gambino,This Is America,Childish Gambino,Golden Hour,Kacey Musgraves,Dua Lipa
61,2019,Bad Guy,Billie Eilish,Bad Guy,Billie Eilish,"When We All Fall Asleep, Where Do We Go?",Billie Eilish,Billie Eilish
62,2020,Everything I Wanted,Billie Eilish,I Can't Breathe,H.E.R.,Folklore,Taylor Swift,Megan Thee Stallion
63,2021,Leave the Door Open,Silk Sonic,Leave the Door Open,Silk Sonic,We Are,Jon Batiste,Olivia Rodrigo


In [164]:
# variable defination
variables_list = [['record of the year', 'artist name record'], 
                  ['song of the year', 'artist name song'],
                  ['album of the year', 'artist name album']]
variable_name = ['record', 'song', 'artist']

# melt from wide to long
for i in range(len(variables_list)):
    var = variables_list[i]
    df_name = 'df_name_{}'.format(variable_name[i])
    df_name = splitted_table_df.melt(id_vars = 'year', value_vars = var, 
                       var_name = 'award', value_name = 'artist')


In [159]:
pd.DataFrame.from_dict(table_dict['year'])

,0
0,\n 1958\n
1,\n 1959\n
2,\n 1960\n
3,\n 1961\n
4,\n 1962\n
...,...
60,\n 2018\n
61,\n 2019\n
62,\n 2020\n
63,\n 2021\n
